Adapting files for MATLAB modeling

In [1]:
import functions_paper_1
import importlib
from functions_paper_1 import *
exec(open("functions_paper_1.py").read())
# Local path to Sharepoint (on your computer, change if you are a different user)
datadir = Path(Path('../datadir_yb.txt').read_text())
data_folder = './datafiles/modeling_'

In [2]:
#Bioreplicates 
# 20240114flow_exp20240111_p2
# 20240120flow_exp20240117_p2
# --
# 20240229flow_exp20240226_p1_BioA
# 20240229flow_exp20240226_p2_BioB
# 20240229flow_exp20240226_p3_BioC

In [3]:
# Import Data - bioreplicate 1 
folder = '20240114flow_exp20240111_p2'
file_path = datadir/'instruments'/'data'/'attune'/'Sneha'/folder/'export_singlets' #Assign file paths
yaml_path = datadir/'instruments'/'data'/'attune'/'Sneha'/folder/'well_metadata.yaml' #Assign yaml paths 
data1 = rd.flow.load_csv_with_metadata(file_path, yaml_path) #Pull data
data1['bioreplicate'] = folder 

In [4]:
# Import Data - bioreplicate 2 
folder = '20240120flow_exp20240117_p2'
file_path = datadir/'instruments'/'data'/'attune'/'Sneha'/folder/'export_singlets' #Assign file paths
yaml_path = datadir/'instruments'/'data'/'attune'/'Sneha'/folder/'well_metadata.yaml' #Assign yaml paths 
data2 = rd.flow.load_csv_with_metadata(file_path, yaml_path) #Pull data
data2['bioreplicate'] = folder 

In [5]:
# Import Data - bioreplicate 4 
folder = '20240229flow_exp20240226_p1_BioA'
file_path = datadir/'instruments'/'data'/'attune'/'Sneha'/folder/'export_singlets' #Assign file paths
yaml_path = datadir/'instruments'/'data'/'attune'/'Sneha'/folder/'well_metadata.yaml' #Assign yaml paths 
data4 = rd.flow.load_csv_with_metadata(file_path, yaml_path) #Pull data
data4['bioreplicate'] = folder 

In [6]:
# Import Data - bioreplicate 5 
folder = '20240229flow_exp20240226_p2_BioB'
file_path = datadir/'instruments'/'data'/'attune'/'Sneha'/folder/'export_singlets' #Assign file paths
yaml_path = datadir/'instruments'/'data'/'attune'/'Sneha'/folder/'well_metadata.yaml' #Assign yaml paths 
data5 = rd.flow.load_csv_with_metadata(file_path, yaml_path) #Pull data
data5['bioreplicate'] = folder 

In [7]:
# Import Data - bioreplicate 6 
folder = '20240229flow_exp20240226_p3_BioC'
file_path = datadir/'instruments'/'data'/'attune'/'Sneha'/folder/'export_singlets' #Assign file paths
yaml_path = datadir/'instruments'/'data'/'attune'/'Sneha'/folder/'well_metadata.yaml' #Assign yaml paths 
data6 = rd.flow.load_csv_with_metadata(file_path, yaml_path) #Pull data
data6['bioreplicate'] = folder 

In [8]:
# simplify df with only necessary stats
col_keep = ['reporter', 'ZF43', 'well', 'mGL-A', 'TagBFP-A', 'iRFP670-A', 'bioreplicate']
data1_subset = data1[col_keep]
data2_subset = data2[col_keep]
data4_subset = data4[col_keep]
data5_subset = data5[col_keep]
data6_subset = data6[col_keep]

In [9]:
# combine all 5 bioreplicate for further trimming
data_subset = pd.concat( [data1_subset, data2_subset, data4_subset, data5_subset, data6_subset], ignore_index=True,join='inner')

In [10]:
data_subset['ZF43'].unique()


array([0.5    , 0.     , 0.03125, 0.01563, 0.0625 , 0.25   , 1.     ,
       0.00781, 0.125  ])

In [11]:
data = data_subset.dropna()

In [12]:
data['ZF43'].unique()

array([0.5    , 0.     , 0.03125, 0.01563, 0.0625 , 0.25   , 1.     ,
       0.00781, 0.125  ])

In [13]:
data['reporter'].unique()

array(['pTET021_203bp 0.1xCre', 'iRFP670', 'pTET021_203bp 0xCre',
       'loxPrept 0xCre', 'loxPrept 0.1xCre', 'unstained'], dtype=object)

In [14]:
data = data[data['reporter'].isin(['pTET021_203bp 0xCre', 'pTET021_203bp 0.1xCre'])]

In [15]:
data.head()

,reporter,ZF43,well,mGL-A,TagBFP-A,iRFP670-A,bioreplicate
0,pTET021_203bp 0.1xCre,0.5,B5,175,1863,37102,20240114flow_exp20240111_p2
1,pTET021_203bp 0.1xCre,0.5,B5,42,-99,105,20240114flow_exp20240111_p2
2,pTET021_203bp 0.1xCre,0.5,B5,227,2003,46942,20240114flow_exp20240111_p2
3,pTET021_203bp 0.1xCre,0.5,B5,1041,2763,72690,20240114flow_exp20240111_p2
4,pTET021_203bp 0.1xCre,0.5,B5,431,1632,30676,20240114flow_exp20240111_p2


In [16]:
# gate by positive values
data = data[ data['TagBFP-A'] >0]
data = data[ data['mGL-A'] >0]
data = data[ data['iRFP670-A'] >0]
data['log10 mGL-A'] = np.log10(data['mGL-A'])

In [17]:
# gate by iRFP670 values
iRFP670_gate = 10000 
data_iRFP670gated = data[ data['iRFP670-A'] > iRFP670_gate]

In [18]:
df = data_iRFP670gated #choose dataframe

#Decide how to group. The last parameter will be the condition that it is group by by last ('well' or 'bioreplicate')
by = ['reporter', 'ZF43','bioreplicate'] #how to filter results, columns 

x = ['mGL-A', 'TagBFP-A'] #which parameter to calculate results on
stat = [sp.stats.gmean] #statistics to calculate
s_bioreplicate = calc_stats(df,by,x,stat) #calculate statistics 

In [19]:
display(s_bioreplicate)

,reporter,ZF43,bioreplicate,mGL-A_gmean,TagBFP-A_gmean,Fraction,Count
0,pTET021_203bp 0.1xCre,0.0,20240114flow_exp20240111_p2,42.240878,34.793400,0.255331,15303
1,pTET021_203bp 0.1xCre,0.0,20240120flow_exp20240117_p2,67.092403,35.144582,0.028949,1735
2,pTET021_203bp 0.1xCre,0.0,20240229flow_exp20240226_p1_BioA,56.497923,37.358951,0.365185,21887
3,pTET021_203bp 0.1xCre,0.0,20240229flow_exp20240226_p2_BioB,48.552109,35.828663,0.230787,13832
4,pTET021_203bp 0.1xCre,0.0,20240229flow_exp20240226_p3_BioC,60.293531,34.340284,0.119748,7177
...,...,...,...,...,...,...,...
85,pTET021_203bp 0xCre,1.0,20240114flow_exp20240111_p2,554.934636,3465.650956,0.055134,10420
86,pTET021_203bp 0xCre,1.0,20240120flow_exp20240117_p2,434.995672,1575.025825,0.054504,10301
87,pTET021_203bp 0xCre,1.0,20240229flow_exp20240226_p1_BioA,352.480854,2476.457948,0.319765,60434
88,pTET021_203bp 0xCre,1.0,20240229flow_exp20240226_p2_BioB,430.749914,2745.918777,0.470764,88972


In [20]:
s_125x_noCre = s_bioreplicate[(s_bioreplicate['ZF43']==0.125)&(s_bioreplicate['reporter']=='pTET021_203bp 0xCre')]
display(s_125x_noCre)

,reporter,ZF43,bioreplicate,mGL-A_gmean,TagBFP-A_gmean,Fraction,Count
70,pTET021_203bp 0xCre,0.125,20240114flow_exp20240111_p2,285.828655,455.527676,0.267518,50886
71,pTET021_203bp 0xCre,0.125,20240120flow_exp20240117_p2,275.133543,222.719350,0.032894,6257
72,pTET021_203bp 0xCre,0.125,20240229flow_exp20240226_p1_BioA,294.471166,292.149333,0.351434,66848
73,pTET021_203bp 0xCre,0.125,20240229flow_exp20240226_p2_BioB,354.705709,312.088396,0.239408,45539
74,pTET021_203bp 0xCre,0.125,20240229flow_exp20240226_p3_BioC,329.508258,295.358451,0.108745,20685


In [21]:
bioreps_plot = pd.array(['20240114flow_exp20240111_p2',
                         '20240120flow_exp20240117_p2',
                         '20240229flow_exp20240226_p1_BioA', 
                         '20240229flow_exp20240226_p2_BioB',
                         '20240229flow_exp20240226_p3_BioC'
                         ])

# initialization for all datapoints
# data_iRFP670gated['Normalized_mGL-A_gmean'] = 0
# data_iRFP670gated['Normalized_mCherry-A_gmean'] = 0

for biorep in bioreps_plot:

    mGL_125x = s_bioreplicate[(s_bioreplicate['bioreplicate']==biorep)&(s_bioreplicate['reporter'] == 'pTET021_203bp 0xCre') & (s_bioreplicate['ZF43'] == 0.125)]['mGL-A_gmean'].iloc[0]
    display(mGL_125x)

    TagBFP_125x = s_bioreplicate[(s_bioreplicate['bioreplicate']==biorep)&(s_bioreplicate['reporter'] == 'pTET021_203bp 0xCre') & (s_bioreplicate['ZF43'] == 0.125)]['TagBFP-A_gmean'].iloc[0]
    display(TagBFP_125x)
    
    # for every data point
    # condition = data_iRFP670gated['bioreplicate'] == biorep
    # data_iRFP670gated.loc[condition, 'Normalized_mGL-A_gmean'] = data_iRFP670gated.loc[condition, 'mGL-A'] / mGL_125x
    # data_iRFP670gated.loc[condition, 'Normalized_mCherry-A_gmean'] = data_iRFP670gated.loc[condition, 'mCherry-A'] / mCherry_125x

    condition = s_bioreplicate['bioreplicate'] == biorep
    s_bioreplicate.loc[condition, 'Normalized_mGL-A_gmean'] = s_bioreplicate.loc[condition, 'mGL-A_gmean'] / mGL_125x
    s_bioreplicate.loc[condition, 'Normalized_TagBFP-A_gmean'] = s_bioreplicate.loc[condition, 'TagBFP-A_gmean'] / TagBFP_125x

285.82865464259334

455.52767643091323

275.1335427638179

222.71935024825757

294.47116592735165

292.1493326072595

354.70570943861713

312.08839570392763

329.50825813699487

295.3584510552382

In [22]:
display(s_bioreplicate)

,reporter,ZF43,bioreplicate,mGL-A_gmean,TagBFP-A_gmean,Fraction,Count,Normalized_mGL-A_gmean,Normalized_TagBFP-A_gmean
0,pTET021_203bp 0.1xCre,0.0,20240114flow_exp20240111_p2,42.240878,34.793400,0.255331,15303,0.147784,0.076380
1,pTET021_203bp 0.1xCre,0.0,20240120flow_exp20240117_p2,67.092403,35.144582,0.028949,1735,0.243854,0.157798
2,pTET021_203bp 0.1xCre,0.0,20240229flow_exp20240226_p1_BioA,56.497923,37.358951,0.365185,21887,0.191862,0.127876
3,pTET021_203bp 0.1xCre,0.0,20240229flow_exp20240226_p2_BioB,48.552109,35.828663,0.230787,13832,0.136880,0.114803
4,pTET021_203bp 0.1xCre,0.0,20240229flow_exp20240226_p3_BioC,60.293531,34.340284,0.119748,7177,0.182980,0.116266
...,...,...,...,...,...,...,...,...,...
85,pTET021_203bp 0xCre,1.0,20240114flow_exp20240111_p2,554.934636,3465.650956,0.055134,10420,1.941494,7.607992
86,pTET021_203bp 0xCre,1.0,20240120flow_exp20240117_p2,434.995672,1575.025825,0.054504,10301,1.581035,7.071796
87,pTET021_203bp 0xCre,1.0,20240229flow_exp20240226_p1_BioA,352.480854,2476.457948,0.319765,60434,1.196996,8.476685
88,pTET021_203bp 0xCre,1.0,20240229flow_exp20240226_p2_BioB,430.749914,2745.918777,0.470764,88972,1.214387,8.798529


In [23]:
data_subset_noCre = s_bioreplicate[s_bioreplicate['reporter']== 'pTET021_203bp 0xCre']
data_subset_Cre = s_bioreplicate[s_bioreplicate['reporter']== 'pTET021_203bp 0.1xCre']

In [24]:
data_subset_noCre['ZF43'].unique()

array([0.     , 0.00781, 0.01563, 0.03125, 0.0625 , 0.125  , 0.25   ,
       0.5    , 1.     ])

In [25]:
data_subset_noCre['ZF43'] = data_subset_noCre['ZF43'].astype(str)
data_subset_Cre['ZF43'] = data_subset_Cre['ZF43'].astype(str)

/var/folders/sg/b0f4ms6d7nx2q59_0mk_kgxm0000gn/T/ipykernel_30129/3671166382.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_subset_noCre['ZF43'] = data_subset_noCre['ZF43'].astype(str)
/var/folders/sg/b0f4ms6d7nx2q59_0mk_kgxm0000gn/T/ipykernel_30129/3671166382.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_subset_Cre['ZF43'] = data_subset_Cre['ZF43'].astype(str)


In [26]:
zf_target_conc = pd.array(['0.0', '0.00781', '0.01563', '0.03125', '0.0625', '0.125', '0.25', '0.5', '1.0'])

# initialization for mean of gmeans
noCre_mean = pd.DataFrame(columns=['zf_conc', 'mean_mGL_noCre', 'mean_TagBFP_noCre'])
Cre_mean = pd.DataFrame(columns=['zf_conc', 'mean_mGL_Cre', 'mean_TagBFP_Cre'])

for zf_conc in zf_target_conc:

    mean_mGL_noCre = data_subset_noCre[data_subset_noCre['ZF43']==zf_conc]['Normalized_mGL-A_gmean'].mean()
    mean_TagBFP_noCre = data_subset_noCre[data_subset_noCre['ZF43']==zf_conc]['Normalized_TagBFP-A_gmean'].mean()

    mean_mGL_Cre = data_subset_Cre[data_subset_Cre['ZF43']==zf_conc]['Normalized_mGL-A_gmean'].mean()
    mean_TagBFP_Cre = data_subset_Cre[data_subset_Cre['ZF43']==zf_conc]['Normalized_TagBFP-A_gmean'].mean()

    noCre_mean.loc[len(noCre_mean)] = [zf_conc, mean_mGL_noCre, mean_TagBFP_noCre]

    Cre_mean.loc[len(Cre_mean)] = [zf_conc, mean_mGL_Cre, mean_TagBFP_Cre]

In [27]:
display(noCre_mean)

,zf_conc,mean_mGL_noCre,mean_TagBFP_noCre
0,0.0,0.138633,0.115477
1,0.00781,0.603113,0.186542
2,0.01563,0.809063,0.259056
3,0.03125,0.861461,0.371134
4,0.0625,0.911086,0.567842
5,0.125,1.000000,1.000000
6,0.25,1.029368,1.808565
7,0.5,1.150555,3.621365
8,1.0,1.415806,7.938978


In [29]:
noCre_file_path = data_folder + 'noCre_ZF43titration.xlsx'
data_subset_noCre.to_excel(noCre_file_path, index=False)

Cre_file_path = data_folder + 'Cre_ZF43titration.xlsx'
data_subset_Cre.to_excel(Cre_file_path, index=False)

In [30]:
mean_noCre_file_path = data_folder + 'MEAN_noCre_ZF43titration.xlsx'
noCre_mean.to_excel(mean_noCre_file_path, index=False)

mean_Cre_file_path = data_folder + 'MEAN_Cre_ZF43titration.xlsx'
Cre_mean.to_excel(mean_Cre_file_path, index=False)

Do calculations for each biorep